Real Private Domestic Final Purchases 

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "pdfp"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [5]:
# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Real PDFP": "LB0000031Q020SBEA",
    "Real GDP": "GDPC1",
}

In [23]:
today = dt.date.today()

real_pdfp = fred.get_series(
    series_id=series_codes["Real PDFP"],
    observation_start=dt.date(2024, 1, 1),
    observation_end=today,
    frequency='q',
    units='pch'
).rename("Real PDFP")

real_gdp = fred.get_series(
    series_id=series_codes["Real GDP"],
    observation_start=dt.date(2024, 1, 1),
    observation_end=today,
    frequency='q',
    units='pch'
).rename("Real GDP")

real_pdfp = real_pdfp * 4
real_gdp = real_gdp * 4


hlb_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "hlb_2.xlsx")

trend_srs = hlb_df.set_index(pd.to_datetime(hlb_df["Date"]))['Trend Growth'].rename("HLW Trend Growth")

trend_srs.tail()


joined_df = pd.concat(
    [real_pdfp, real_gdp, trend_srs], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df.tail()

,Real PDFP,Real GDP,HLW Trend Growth
Date,,,
2024-04-01,3.32840,3.54196,2.432907
2024-07-01,3.40664,3.29912,2.471950
2024-10-01,2.63524,1.83948,2.449158
2025-01-01,1.89188,-0.65008,2.336189
2025-04-01,2.83224,3.78400,2.450188


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [27]:
import numpy as np

# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Real GDP & PDPF<br><sup>Quarterly, Annualized, Seasonally Adjusted </sup>'),
    xaxis_title="Date",
    yaxis_title="Change (%)",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%",
    ticksuffix="%",
    range=[-1.1, 5]
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    dtick='M12',
    tickformat='%Y',
    title_text=None,
)

# Add a dotted line projecting Real GDP to the next quarter at 3.8

# # Get the last date in the index and add one quarter
# last_date = joined_df.index[-1]
# next_quarter = (last_date + pd.offsets.QuarterEnd())

# # Coordinates for the projection
# x_proj = [last_date, next_quarter]
# y_proj = [joined_df["Real GDP"].iloc[-1], 3.9]

# fig.add_trace(
#     go.Scatter(
#         x=x_proj,
#         y=y_proj,
#         mode='lines+markers',
#         name='Real GDP Projection',
#         line=dict(dash='dot', color='#CF8B40'),
#         marker=dict(color='#CF8B40', size=8, symbol='circle'),
#         showlegend=False,
#     )
# )

# # Add annotation above the projected point
# fig.add_annotation(
#     x=next_quarter,
#     y=3.5,
#     text="GDPNow (10/14)",
#     showarrow=False,
#     yanchor="bottom",
#     xanchor="left",
#     xshift=6,
#     font=dict(color="#CF8B40")
# )

fig.update_layout(
    height=550,
    width=1150,
)

fig.add_hline(
    y=1.8,
    line_dash="dash",
    line_color="red",
    annotation_text="Sep. SEP Longer-Run Projection: 1.8%</br>",
    annotation_position="bottom right"
)

ut.add_end_labels(fig = fig, offset=0.05)

fig.update_layout(legend=dict(font=dict(size=20)))

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")